## Running REST2 on vanilla alanine system results in dihedrals getting stuck... check to see if something is wrong with the REST2 implementation

In [1]:
import pickle
import os
from perses.annihilation.rest import RESTTopologyFactory
from perses.annihilation.lambda_protocol import RESTState
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from openmmtools import cache, utils
from perses.dispersed.utils import configure_platform
cache.global_context_cache.platform = configure_platform(utils.get_fastest_platform().getName())
from simtk import openmm, unit
import math
from openmmtools.constants import kB
from openmmtools import mcmc, multistate
import argparse
import copy
from perses.dispersed import feptasks
import numpy as np
from simtk.openmm import app
from openmmforcefields.generators import SystemGenerator
import pickle
import mdtraj as md
import itertools

INFO:numexpr.utils:Note: detected 72 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 72 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


In [2]:
pdb = app.PDBFile("../../input/ala_vacuum.pdb")

In [3]:
forcefield_files = ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
barostat = openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 298 * unit.kelvin, 50)
system_generator = SystemGenerator(forcefields=forcefield_files,
                               barostat=barostat,
                               forcefield_kwargs={'removeCMMotion': False,
                                                    'ewaldErrorTolerance': 1e-4,
                                                    'constraints' : app.HBonds,
                                                    'hydrogenMass' : 4 * unit.amus},
                                periodic_forcefield_kwargs={'nonbondedMethod': app.PME},
                                small_molecule_forcefield='gaff-2.11',
                                nonperiodic_forcefield_kwargs=None, 
                                   molecules=None, 
                                   cache=None)
modeller = app.Modeller(pdb.topology, pdb.positions)
modeller.addSolvent(system_generator.forcefield, model='tip3p', padding=9*unit.angstroms, ionicStrength=0.15*unit.molar)
solvated_topology = modeller.getTopology()
solvated_positions = modeller.getPositions()

# Canonicalize the solvated positions: turn tuples into np.array
positions = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in solvated_positions.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)
sys = system_generator.create_system(solvated_topology)



DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11


In [4]:
for atom in solvated_topology.atoms():
    print(atom)

<Atom 0 (H1) of chain 0 residue 0 (ACE)>
<Atom 1 (CH3) of chain 0 residue 0 (ACE)>
<Atom 2 (H2) of chain 0 residue 0 (ACE)>
<Atom 3 (H3) of chain 0 residue 0 (ACE)>
<Atom 4 (C) of chain 0 residue 0 (ACE)>
<Atom 5 (O) of chain 0 residue 0 (ACE)>
<Atom 6 (N) of chain 0 residue 1 (ALA)>
<Atom 7 (H) of chain 0 residue 1 (ALA)>
<Atom 8 (CA) of chain 0 residue 1 (ALA)>
<Atom 9 (HA) of chain 0 residue 1 (ALA)>
<Atom 10 (CB) of chain 0 residue 1 (ALA)>
<Atom 11 (HB2) of chain 0 residue 1 (ALA)>
<Atom 12 (HB3) of chain 0 residue 1 (ALA)>
<Atom 13 (C) of chain 0 residue 1 (ALA)>
<Atom 14 (O) of chain 0 residue 1 (ALA)>
<Atom 15 (HB1) of chain 0 residue 1 (ALA)>
<Atom 16 (N) of chain 0 residue 2 (NME)>
<Atom 17 (H) of chain 0 residue 2 (NME)>
<Atom 18 (C) of chain 0 residue 2 (NME)>
<Atom 19 (H1) of chain 0 residue 2 (NME)>
<Atom 20 (H2) of chain 0 residue 2 (NME)>
<Atom 21 (H3) of chain 0 residue 2 (NME)>
<Atom 22 (O) of chain 1 residue 3 (HOH)>
<Atom 23 (H1) of chain 1 residue 3 (HOH)>
<Atom 24

In [4]:
# Build REST factory
factory = RESTTopologyFactory(sys, solute_region=list(range(22)))

# Get REST system
REST_system = factory.REST_system


INFO:REST:Added MonteCarloBarostat.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.5648, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.5648, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.5648), unit=nanometer)]
INFO:REST:No unknown forces.


In [5]:
T_max = 600.0

# Create states for each replica
n_replicas = 12  # Number of temperature replicas.
T_min = 298.0 * unit.kelvin  # Minimum temperature.
T_max = T_max * unit.kelvin  # Maximum temperature.
temperatures = [T_min + (T_max - T_min) * (math.exp(float(i) / float(n_replicas-1)) - 1.0) / (math.e - 1.0)
                for i in range(n_replicas)]

# Create reference thermodynamic state
lambda_zero_alchemical_state = RESTState.from_system(REST_system)
thermostate = ThermodynamicState(REST_system, temperature=T_min)
compound_thermodynamic_state = CompoundThermodynamicState(thermostate, composable_states=[lambda_zero_alchemical_state])

# Create thermodynamics states
sampler_state =  SamplerState(positions, box_vectors=sys.getDefaultPeriodicBoxVectors())
beta_0 = 1/(kB*T_min)
thermodynamic_state_list = []
sampler_state_list = []
for temperature in temperatures:
    beta_m = 1/(kB*temperature)
    compound_thermodynamic_state_copy = copy.deepcopy(compound_thermodynamic_state)
    compound_thermodynamic_state_copy.set_alchemical_parameters(beta_0, beta_m)
    thermodynamic_state_list.append(compound_thermodynamic_state_copy)

    # now generating a sampler_state for each thermodyanmic state, with relaxed positions
#     context, context_integrator = context_cache.get_context(compound_thermodynamic_state_copy) # this line doesn't do anything
    feptasks.minimize(compound_thermodynamic_state_copy, sampler_state)
    sampler_state_list.append(copy.deepcopy(sampler_state))


In [6]:
# Save REST energy
thermodynamic_state_list[11].reduced_potential(sampler_state_list[11])

-8791.524228371154

In [15]:
compound_thermodynamic_state_copy.solute_scale

0.4966666666666667

In [16]:
compound_thermodynamic_state_copy.inter_scale

0.7047458170621992

In [17]:
for i in range(REST_system.getNumForces()):
    print(REST_system.getForce(i))

<simtk.openmm.openmm.MonteCarloBarostat; proxy of <Swig Object of type 'OpenMM::MonteCarloBarostat *' at 0x2b8642b85d80> >
<simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2b864294c2d0> >
<simtk.openmm.openmm.CustomAngleForce; proxy of <Swig Object of type 'OpenMM::CustomAngleForce *' at 0x2b864294c2d0> >
<simtk.openmm.openmm.CustomTorsionForce; proxy of <Swig Object of type 'OpenMM::CustomTorsionForce *' at 0x2b864294c2d0> >
<simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2b864294c2d0> >
<simtk.openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x2b864294c2d0> >
<simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2b864294c2d0> >


In [18]:
last_state = thermodynamic_state_list[11]

In [19]:
last_state.temperature

Quantity(value=600.0, unit=kelvin)

In [20]:
last_state_system = last_state.get_system()

In [21]:
for i in range(last_state_system.getNumForces()):
    print(last_state_system.getForce(i))

<simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2b86427c1090> >
<simtk.openmm.openmm.CustomAngleForce; proxy of <Swig Object of type 'OpenMM::CustomAngleForce *' at 0x2b864294c3c0> >
<simtk.openmm.openmm.CustomTorsionForce; proxy of <Swig Object of type 'OpenMM::CustomTorsionForce *' at 0x2b864294c3c0> >
<simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2b864294c3c0> >
<simtk.openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x2b864294c3c0> >
<simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2b864294c3c0> >
<simtk.openmm.openmm.AndersenThermostat; proxy of <Swig Object of type 'OpenMM::AndersenThermostat *' at 0x2b864294c3c0> >
<simtk.openmm.openmm.MonteCarloBarostat; proxy of <Swig Object of type 'OpenMM::MonteCarloBarostat *' at 0x2b864294c3c0> >


In [22]:
first_state_system = thermodynamic_state_list[0].get_system()

In [23]:
thermodynamic_state_list[0].solute_scale

1.0

In [24]:
thermodynamic_state_list[0].inter_scale

1.0

### Check terms of protein-environment region

In [7]:
# Compute energy for non-RESTified system
# Determine regions and scaling factors
protein = list(range(22))
environment = list(range(22, solvated_topology.getNumAtoms()))
beta_0 = 1/(kB*T_min)
beta_m = 1/(kB*T_max)
protein_scaling = beta_m / beta_0
inter_scaling = np.sqrt(beta_m / beta_0)


In [28]:
# Check the terms in the bond force
bond_force = last_state_system.getForce(0)
bond_force_reference = first_state_system.getForce(0)
for bond in range(bond_force.getNumBonds()):
    p1, p2, params = bond_force.getBondParameters(bond)
    print(p1, p2, params)
    print(bond_force_reference.getBondParameters(bond))
    print()
#     if p1 in protein and p2 in protein:
#         bond_force.setBondParameters(bond, p1, p2, length, k * protein_scaling)
#     if (p1 in protein and p2 in environment) or (p1 in environment and p2 in protein):
#         print (p1, p2, params)
#         print(bond_force_reference.getBondParameters(bond))
#         print()


4 1 (0.1522, 265265.6, 0.0)
[4, 1, (0.1522, 265265.6, 0.0)]

4 5 (0.12290000000000001, 476975.9999999999, 0.0)
[4, 5, (0.12290000000000001, 476975.9999999999, 0.0)]

4 6 (0.1335, 410031.99999999994, 0.0)
[4, 6, (0.1335, 410031.99999999994, 0.0)]

13 8 (0.1522, 265265.6, 0.0)
[13, 8, (0.1522, 265265.6, 0.0)]

13 14 (0.12290000000000001, 476975.9999999999, 0.0)
[13, 14, (0.12290000000000001, 476975.9999999999, 0.0)]

8 10 (0.1526, 259407.99999999994, 0.0)
[8, 10, (0.1526, 259407.99999999994, 0.0)]

8 6 (0.1449, 282001.5999999999, 0.0)
[8, 6, (0.1449, 282001.5999999999, 0.0)]

13 16 (0.1335, 410031.99999999994, 0.0)
[13, 16, (0.1335, 410031.99999999994, 0.0)]

18 16 (0.1449, 282001.5999999999, 0.0)
[18, 16, (0.1449, 282001.5999999999, 0.0)]



In [27]:
for i in range(bond_force.getNumPerBondParameters()):
    print(bond_force.getPerBondParameterName(i))


length
k
identifier


--> There are no bonds to hydrogen here??

In [26]:
# Check the terms in the angle force
angle_force = last_state_system.getForce(1)
angle_force_reference = first_state_system.getForce(1)
for angle_index in range(angle_force.getNumAngles()):
    p1, p2, p3, params = angle_force.getAngleParameters(angle_index)
    print(p1, p2, p3, params)


0 1 2 (1.911135530933791, 292.88, 2.0)
0 1 3 (1.911135530933791, 292.88, 2.0)
0 1 4 (1.911135530933791, 418.40000000000003, 2.0)
1 4 5 (2.101376419401173, 669.44, 2.0)
1 4 6 (2.035053907825388, 585.76, 2.0)
2 1 3 (1.911135530933791, 292.88, 2.0)
2 1 4 (1.911135530933791, 418.40000000000003, 2.0)
3 1 4 (1.911135530933791, 418.40000000000003, 2.0)
4 6 7 (2.0943951023931953, 418.40000000000003, 2.0)
4 6 8 (2.1275563581810877, 418.40000000000003, 2.0)
5 4 6 (2.1450096507010312, 669.44, 2.0)
6 8 9 (1.911135530933791, 418.40000000000003, 2.0)
6 8 10 (1.9146261894377796, 669.44, 2.0)
6 8 13 (1.9216075064457567, 527.184, 2.0)
7 6 8 (2.0601866490541068, 418.40000000000003, 2.0)
8 10 11 (1.911135530933791, 418.40000000000003, 2.0)
8 10 12 (1.911135530933791, 418.40000000000003, 2.0)
8 10 15 (1.911135530933791, 418.40000000000003, 2.0)
8 13 14 (2.101376419401173, 669.44, 2.0)
8 13 16 (2.035053907825388, 585.76, 2.0)
9 8 10 (1.911135530933791, 418.40000000000003, 2.0)
9 8 13 (1.911135530933791, 41

--> angles look good

In [ ]:

# # Check the terms in the torsion force
# torsion_force = system.getForce(2)
# for torsion_index in range(torsion_force.getNumTorsions()):
#     p1, p2, p3, p4, periodicity, phase, k = torsion_force.getTorsionParameters(torsion_index)
#     if p1 in protein and p2 in protein and p3 in protein and p4 in protein:
#         torsion_force.setTorsionParameters(torsion_index, p1, p2, p3, p4, periodicity, phase, k * protein_scaling)
#     elif set([p1, p2, p3, p4]).intersection(set(protein)) != set() and set([p1, p2, p3, p4]).intersection(set(environment)) != set():
#         torsion_force.setTorsionParameters(torsion_index, p1, p2, p3, p4, periodicity, phase, k * inter_scaling)

# # Check the terms in the nonbonded force
# nb_force = system.getForce(3)
# for nb_index in range(nb_force.getNumParticles()):
#     charge, sigma, epsilon = nb_force.getParticleParameters(nb_index)
#     if nb_index in protein:
#         nb_force.setParticleParameters(nb_index, protein_scaling * charge, sigma, protein_scaling * epsilon)

# # Check the exceptions in the nonbonded force
# for nb_index in range(nb_force.getNumExceptions()):
#     p1, p2, chargeProd, sigma, epsilon = nb_force.getExceptionParameters(nb_index)
#     if p1 in protein and p2 in protein:
#         nb_force.setExceptionParameters(nb_index, p1, p2, protein_scaling * chargeProd, sigma, protein_scaling * epsilon)
#     elif (p1 in protein and p2 in environment) or (p1 in environment and p2 in protein):
#         nb_force.setExceptionParameters(nb_index, p1, p2, inter_scaling * chargeProd, sigma, inter_scaling * epsilon)


In [8]:
# Manually scale terms down

T = 600.0* unit.kelvin

# Set alchemical parameters
beta_0 = 1 / (kB * T_min)
beta_m = 1 / (kB * T)

# Compute energy for non-RESTified system
# Determine regions and scaling factors
protein = range(0, 22)
environment = list(range(22, sys.getNumParticles()))
protein_scaling = beta_m / beta_0
inter_scaling = np.sqrt(beta_m / beta_0)

system = sys

# Scale the terms in the bond force appropriately
bond_force = system.getForce(0)
for bond in range(bond_force.getNumBonds()):
    p1, p2, length, k = bond_force.getBondParameters(bond)
    if p1 in protein and p2 in protein:
        bond_force.setBondParameters(bond, p1, p2, length, k * protein_scaling)
    elif (p1 in protein and p2 in environment) or (p1 in environment and p2 in protein):
        bond_force.setBondParameters(bond, p1, p2, length, k * inter_scaling)

# Scale the terms in the angle force appropriately
angle_force = system.getForce(1)
for angle_index in range(angle_force.getNumAngles()):
    p1, p2, p3, angle, k = angle_force.getAngleParameters(angle_index)
    if p1 in protein and p2 in protein and p3 in protein:
        angle_force.setAngleParameters(angle_index, p1, p2, p3, angle, k * protein_scaling)
    elif set([p1, p2, p3]).intersection(set(protein)) != set() and set([p1, p2, p3]).intersection(set(environment)) != set():
        angle_force.setAngleParameters(angle_index, p1, p2, p3, angle, k * inter_scaling)

# Scale the terms in the torsion force appropriately
torsion_force = system.getForce(2)
for torsion_index in range(torsion_force.getNumTorsions()):
    p1, p2, p3, p4, periodicity, phase, k = torsion_force.getTorsionParameters(torsion_index)
    if p1 in protein and p2 in protein and p3 in protein and p4 in protein:
        torsion_force.setTorsionParameters(torsion_index, p1, p2, p3, p4, periodicity, phase, k * protein_scaling)
    elif set([p1, p2, p3, p4]).intersection(set(protein)) != set() and set([p1, p2, p3, p4]).intersection(set(environment)) != set():
        torsion_force.setTorsionParameters(torsion_index, p1, p2, p3, p4, periodicity, phase, k * inter_scaling)

# Scale the terms in the nonbonded force appropriately
nb_force = system.getForce(3)
for nb_index in range(nb_force.getNumParticles()):
    charge, sigma, epsilon = nb_force.getParticleParameters(nb_index)
    if nb_index in protein:
        nb_force.setParticleParameters(nb_index, protein_scaling * charge, sigma, protein_scaling * epsilon)

# Scale the exceptions in the nonbonded force appropriately
for nb_index in range(nb_force.getNumExceptions()):
    p1, p2, chargeProd, sigma, epsilon = nb_force.getExceptionParameters(nb_index)
    if p1 in protein and p2 in protein:
        nb_force.setExceptionParameters(nb_index, p1, p2, protein_scaling * chargeProd, sigma, protein_scaling * epsilon)
    elif (p1 in protein and p2 in environment) or (p1 in environment and p2 in protein):
        nb_force.setExceptionParameters(nb_index, p1, p2, inter_scaling * chargeProd, sigma, inter_scaling * epsilon)



In [9]:
# Relax and save energy
thermostate = ThermodynamicState(system, temperature=T_min)
# context_cache = cache.ContextCache()
# context, context_integrator = context_cache.get_context(thermostate)
thermostate.reduced_potential(sampler_state)



-8791.524228371154

In [14]:
protein_scaling

0.4966666666666667

In [16]:
inter_scaling

0.7047458170621992

In [ ]:
thermostate.

## Build system with a harmonic bond force only

In [1]:
import pickle
import os
from perses.annihilation.rest import RESTTopologyFactory
from perses.annihilation.lambda_protocol import RESTState
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from openmmtools import cache, utils
from perses.dispersed.utils import configure_platform
cache.global_context_cache.platform = configure_platform(utils.get_fastest_platform().getName())
from simtk import openmm, unit
import math
from openmmtools.constants import kB
from openmmtools import mcmc, multistate
import argparse
import copy
from perses.dispersed import feptasks
import numpy as np
from simtk.openmm import app
from openmmforcefields.generators import SystemGenerator
import pickle
import mdtraj as md
import itertools

INFO:numexpr.utils:Note: detected 72 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 72 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


Then added in angle and torsion forces without problem. After adding nonbonded back in, there was a difference in the energies.

In [2]:
# pdb = app.PDBFile("../../input/ala_vacuum.pdb")

# forcefield_files = ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
# barostat = openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 298 * unit.kelvin, 50)
# system_generator = SystemGenerator(forcefields=forcefield_files,
#                                barostat=barostat,
#                                forcefield_kwargs={'removeCMMotion': False,
#                                                     'ewaldErrorTolerance': 1e-4,
#                                                     'constraints' : app.HBonds,
#                                                     'hydrogenMass' : 4 * unit.amus},
#                                 periodic_forcefield_kwargs={'nonbondedMethod': app.PME},
#                                 small_molecule_forcefield='gaff-2.11',
#                                 nonperiodic_forcefield_kwargs=None, 
#                                    molecules=None, 
#                                    cache=None)
# modeller = app.Modeller(pdb.topology, pdb.positions)
# modeller.addSolvent(system_generator.forcefield, model='tip3p', padding=9*unit.angstroms, ionicStrength=0.15*unit.molar)
# solvated_topology = modeller.getTopology()
# solvated_positions = modeller.getPositions()

# # Canonicalize the solvated positions: turn tuples into np.array
# positions = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in solvated_positions.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)
# sys = system_generator.create_system(solvated_topology)



DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11


In [2]:
from openmmtools.testsystems import AlanineDipeptideExplicit
ala = AlanineDipeptideExplicit()
ala.system.removeForce(4)

In [4]:
ala.system.getForces()

[<simtk.openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x2b09a062ede0> >,
 <simtk.openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x2b09a062efc0> >,
 <simtk.openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x2b09a062ec60> >,
 <simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2b09a062ef00> >]

In [9]:
for atom in ala.topology.atoms():
    print(atom)

<Atom 0 (H1) of chain 0 residue 0 (ACE)>
<Atom 1 (CH3) of chain 0 residue 0 (ACE)>
<Atom 2 (H2) of chain 0 residue 0 (ACE)>
<Atom 3 (H3) of chain 0 residue 0 (ACE)>
<Atom 4 (C) of chain 0 residue 0 (ACE)>
<Atom 5 (O) of chain 0 residue 0 (ACE)>
<Atom 6 (N) of chain 0 residue 1 (ALA)>
<Atom 7 (H) of chain 0 residue 1 (ALA)>
<Atom 8 (CA) of chain 0 residue 1 (ALA)>
<Atom 9 (HA) of chain 0 residue 1 (ALA)>
<Atom 10 (CB) of chain 0 residue 1 (ALA)>
<Atom 11 (HB1) of chain 0 residue 1 (ALA)>
<Atom 12 (HB2) of chain 0 residue 1 (ALA)>
<Atom 13 (HB3) of chain 0 residue 1 (ALA)>
<Atom 14 (C) of chain 0 residue 1 (ALA)>
<Atom 15 (O) of chain 0 residue 1 (ALA)>
<Atom 16 (N) of chain 0 residue 2 (NME)>
<Atom 17 (H) of chain 0 residue 2 (NME)>
<Atom 18 (C) of chain 0 residue 2 (NME)>
<Atom 19 (H1) of chain 0 residue 2 (NME)>
<Atom 20 (H2) of chain 0 residue 2 (NME)>
<Atom 21 (H3) of chain 0 residue 2 (NME)>
<Atom 22 (O) of chain 0 residue 3 (HOH)>
<Atom 23 (H1) of chain 0 residue 3 (HOH)>
<Atom 24

In [3]:
for i in range(sys.getNumForces()):
    print(sys.getForce(i))

<simtk.openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x2b40d5c03420> >
<simtk.openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x2b40d5c03840> >
<simtk.openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x2b40d5c03420> >
<simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2b40d5c03840> >
<simtk.openmm.openmm.MonteCarloBarostat; proxy of <Swig Object of type 'OpenMM::MonteCarloBarostat *' at 0x2b40d5c03420> >


In [12]:
ala.system.getForce(3).getSwitchingDistance()

Quantity(value=0.8500000000000001, unit=nanometer)

In [4]:
# for i in range(sys.getForce(3).getNumParticles()):
#     print(i, sys.getForce(3).getParticleParameters(i))

In [5]:
# protein = list(range(0, 22))
# environment = list(range(22, sys.getNumParticles()))

# for i in range(sys.getForce(3).getNumExceptions()):
# #     print(i, sys.getForce(3).getExceptionParameters(i))
#     p1, p2, chargeProd, sigma, epsilon = sys.getForce(3).getExceptionParameters(i)
#     if p1 in protein and p2 in protein:
# #         nb_force.setExceptionParameters(nb_index, p1, p2, protein_scaling * chargeProd, sigma, protein_scaling * epsilon)
#         print("solute ", p1, p2)
#     elif (p1 in protein and p2 in environment) or (p1 in environment and p2 in protein):
# #         nb_force.setExceptionParameters(nb_index, p1, p2, inter_scaling * chargeProd, sigma, inter_scaling * epsilon)
#         print("inter ", p1, p2)

In [3]:
# toy_system = openmm.System()


In [4]:
# for particle_idx in range(sys.getNumParticles()):
#     particle_mass = sys.getParticleMass(particle_idx)
#     hybrid_idx = toy_system.addParticle(particle_mass)

# # barostat = copy.deepcopy(sys.getForce(4))
# # toy_system.addForce(barostat)

# # Copy over the box vectors:
# box_vectors = sys.getDefaultPeriodicBoxVectors()
# toy_system.setDefaultPeriodicBoxVectors(*box_vectors)


In [5]:
# for constraint_idx in range(sys.getNumConstraints()):
#     atom1, atom2, length = sys.getConstraintParameters(constraint_idx)
#     toy_system.addConstraint(atom1, atom2, length)


In [6]:
# harmonic_bond_force = copy.deepcopy(sys.getForce(0))
# toy_system.addForce(harmonic_bond_force)

# harmonic_angle_force = copy.deepcopy(sys.getForce(1))
# toy_system.addForce(harmonic_angle_force)

# torsion_force = copy.deepcopy(sys.getForce(2))
# toy_system.addForce(torsion_force)

# # nb_force = copy.deepcopy(sys.getForce(3))
# # toy_system.addForce(nb_force)


2

In [3]:
class REST2(RESTTopologyFactory):
    _known_forces = {'HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'}

    def __init__(self, system, solute_region, use_dispersion_correction=False):
        """
        arguments
            system : simtk.openmm.system
                system that will be rewritten
            solute_region : simtk.openmm.system
                subset solute region
            use_dispersion_correction : bool, default False
                whether to use a dispersion correction

        Properties
        ----------
            REST_system : simtk.openmm.system
                the REST2-implemented system
        """
        self._use_dispersion_correction = use_dispersion_correction
        self._num_particles = system.getNumParticles()
        self._og_system = system
        self._og_system_forces = {type(force).__name__ : force for force in self._og_system.getForces()}
        self._out_system_forces = {}
        self._solute_region = solute_region
        self._solvent_region = list(set(range(self._num_particles)).difference(set(self._solute_region)))

        assert set(solute_region).issubset(set(range(self._num_particles))), f"the solute region is not a subset of the system particles"
        self._nonbonded_method = self._og_system_forces['NonbondedForce'].getNonbondedMethod()
        self._out_system = openmm.System()

        for particle_idx in range(self._num_particles):
            particle_mass = self._og_system.getParticleMass(particle_idx)
            hybrid_idx = self._out_system.addParticle(particle_mass)

        if "MonteCarloBarostat" in self._og_system_forces.keys():
            barostat = copy.deepcopy(self._og_system_forces["MonteCarloBarostat"])
            self._out_system.addForce(barostat)
            self._out_system_forces[barostat.__class__.__name__] = barostat

        # Copy over the box vectors:
        box_vectors = self._og_system.getDefaultPeriodicBoxVectors()
        self._out_system.setDefaultPeriodicBoxVectors(*box_vectors)

        self._og_system_exceptions = self._generate_dict_from_exceptions(self._og_system_forces['NonbondedForce'])


#         # Check that there are no unknown forces in the new and old systems:
#         for system_name in ['og']:
#             force_names = getattr(self, '_{}_system_forces'.format(system_name)).keys()
#             unknown_forces = set(force_names) - set(self._known_forces)
#             if len(unknown_forces) > 0:
#                 raise ValueError("Unkown forces {} encountered in {} system" % (unknown_forces, system_name))
#         _logger.info("No unknown forces.")

        self._handle_constraints()

        self._add_bond_force_terms()
        self._add_bonds()

        self._add_angle_force_terms()
        self._add_angles()

        self._add_torsion_force_terms()
        self._add_torsions()

        self._add_nonbonded_force_terms()
        self._add_nonbondeds()
        
    def _add_nonbondeds(self):
        self._solute_exceptions, self._interexceptions = [], []

        #the output nonbonded force _only_ contains solvent atoms (the rest are zeroed); same with exceptions
        """
        First, handle the NonbondedForce in the out_system
        """
        og_nb_force = self._og_system_forces['NonbondedForce']
        for particle_idx in range(self._num_particles):
            q, sigma, epsilon = og_nb_force.getParticleParameters(particle_idx)
            identifier = self.get_identifier(particle_idx)

            if identifier == 1:
#                 print("1, ", particle_idx, q, sigma, epsilon, identifier)
                self._out_system_forces['NonbondedForce'].addParticle(q, sigma, epsilon)
                self._out_system_forces['CustomNonbondedForce'].addParticle([q, sigma, epsilon, identifier])
            else:
                self._out_system_forces['NonbondedForce'].addParticle(q*0.0, sigma, epsilon*0.0)
#                 print("0, ", particle_idx, q, sigma, epsilon, identifier)
                self._out_system_forces['CustomNonbondedForce'].addParticle([q, sigma, epsilon, identifier])

        #add appropriate interaction group
        solute_ig, solvent_ig = set(self._solute_region), set(self._solvent_region)
        self._out_system_forces['CustomNonbondedForce'].addInteractionGroup(solute_ig, solvent_ig)
        self._out_system_forces['CustomNonbondedForce'].addInteractionGroup(solute_ig, solute_ig)

        #handle exceptions
        for exception_idx in range(og_nb_force.getNumExceptions()):
            p1, p2, chargeProd, sigma, epsilon = og_nb_force.getExceptionParameters(exception_idx)
            identifier = self.get_identifier([p1, p2])
            if identifier == 1:
                self._out_system_forces['NonbondedForce'].addException(p1, p2, chargeProd, sigma, epsilon)
                self._out_system_forces['CustomNonbondedForce'].addExclusion(p1, p2) #maintain consistent exclusions w/ exceptions
            elif identifier == 0:
                self._solute_exceptions.append([p1, p2, [chargeProd, sigma, epsilon]])
                self._out_system_forces['NonbondedForce'].addException(p1, p2, chargeProd*0.0, sigma, epsilon*0.0)
                self._out_system_forces['CustomNonbondedForce'].addExclusion(p1, p2) #maintain consistent exclusions w/ exceptions
            elif identifier == 2:
                self._interexceptions.append([p1, p2, [chargeProd, sigma, epsilon]])
                self._out_system_forces['NonbondedForce'].addException(p1, p2, chargeProd*0.0, sigma, epsilon*0.0)
                self._out_system_forces['CustomNonbondedForce'].addExclusion(p1, p2) #maintain consistent exclusions w/ exceptions

        #now add the CustomBondForce for exceptions
        exception_force = self._out_system_forces['CustomExceptionForce']

        for solute_exception_term in self._solute_exceptions:
            p1, p2, [chargeProd, sigma, epsilon] = solute_exception_term
            if (chargeProd.value_in_unit_system(unit.md_unit_system) != 0.0) or (epsilon.value_in_unit_system(unit.md_unit_system) != 0.0):
                identifier = 0
                exception_force.addBond(p1, p2, [chargeProd, sigma, epsilon, identifier])

        for interexception_term in self._interexceptions:
            p1, p2, [chargeProd, sigma, epsilon] = interexception_term
            if (chargeProd.value_in_unit_system(unit.md_unit_system) != 0.0) or (epsilon.value_in_unit_system(unit.md_unit_system) != 0.0):
                identifier = 2
                exception_force.addBond(p1, p2, [chargeProd, sigma, epsilon, identifier])

#     def scaling_expression(self, nb=False):
#         if not nb:
# #             last_line = '_is_solute = delta(identifier); _is_solvent = delta(1-identifier); _is_inter = delta(2-identifier);'
# #             out = f"scale_factor = solute_scale*_is_solute + solvent_scale*_is_solvent + inter_scale*_is_inter; \
# #                solvent_scale = 1.; \
# #                {last_line} \
# #                 "
#             out = f"scale_factor = solvent_scale; \
#                solvent_scale = 1.;"
#         else:
#             print("getting into the nb version")
# #             last_line = '_is_solute = delta(identifier1 + identifier2); _is_solvent = delta(2 - (identifier1 + identifier2)); _is_inter = delta(1 - (identifier1 + identifier2));'
# #             last_line = '_is_inter = select(identifier1+identifier2-1, 0, 1); _is_solute = select(identifier1+identifier2, 0, 1); _is_solvent = select(identifier1 + identifier2 - 2, 0, 1);'
# #             last_line = '_is_inter = delta(1); _is_solute = delta(1); _is_solvent = delta(0);'

#             out = f"scale_factor = solvent_scale; \
#                    solvent_scale = 1.; "
#         return out            
    
    
    def _add_nonbonded_force_terms(self):
        from openmmtools.constants import ONE_4PI_EPS0 # OpenMM constant for Coulomb interactions (implicitly in md_unit_system units)

        standard_nonbonded_force = openmm.NonbondedForce()
        custom_nonbonded_expression = f"(4*epsilon*((sigma/r)^12-(sigma/r)^6) + ONE_4PI_EPS0*chargeProd/r) * scale_factor; \
                                        sigma=0.5*(sigma1+sigma2); \
                                        epsilon=sqrt(epsilon1*epsilon2); \
                                        ONE_4PI_EPS0 = {ONE_4PI_EPS0}; \
                                        chargeProd=q1*q2;"

        custom_nonbonded_expression += self.scaling_expression(nb=True)
        custom_nonbonded_force = openmm.CustomNonbondedForce(custom_nonbonded_expression)
        
        custom_nonbonded_force.addPerParticleParameter("q")
        custom_nonbonded_force.addPerParticleParameter("sigma")
        custom_nonbonded_force.addPerParticleParameter("epsilon")
        custom_nonbonded_force.addPerParticleParameter("identifier")

        custom_nonbonded_force.addGlobalParameter('solute_scale', 1.0)
        custom_nonbonded_force.addGlobalParameter('inter_scale', 1.0)

        #set the appropriate parameters
        epsilon_solvent = self._og_system_forces['NonbondedForce'].getReactionFieldDielectric()
        r_cutoff = self._og_system_forces['NonbondedForce'].getCutoffDistance()
        if self._nonbonded_method != openmm.NonbondedForce.NoCutoff:
            standard_nonbonded_force.setReactionFieldDielectric(epsilon_solvent)
            standard_nonbonded_force.setCutoffDistance(r_cutoff)
            custom_nonbonded_force.setCutoffDistance(r_cutoff)
        if self._nonbonded_method in [openmm.NonbondedForce.PME, openmm.NonbondedForce.Ewald]:
            [alpha_ewald, nx, ny, nz] = self._og_system_forces['NonbondedForce'].getPMEParameters()
            delta = self._og_system_forces['NonbondedForce'].getEwaldErrorTolerance()
            standard_nonbonded_force.setPMEParameters(alpha_ewald, nx, ny, nz)
            standard_nonbonded_force.setEwaldErrorTolerance(delta)
        standard_nonbonded_force.setNonbondedMethod(self._nonbonded_method)
        custom_nonbonded_force.setNonbondedMethod(self._translate_nonbonded_method_to_custom(self._nonbonded_method))

        #translate nonbonded to custom
        if self._og_system_forces['NonbondedForce'].getUseDispersionCorrection():
            standard_nonbonded_force.setUseDispersionCorrection(True)
            if self._use_dispersion_correction:
                custom_nonbonded_force.setUseLongRangeCorrection(True)
        else:
            custom_nonbonded_force.setUseLongRangeCorrection(False)

        if self._og_system_forces['NonbondedForce'].getUseSwitchingFunction():
            switching_distance = self._og_system_forces['NonbondedForce'].getSwitchingDistance()
            standard_nonbonded_force.setUseSwitchingFunction(True)
            standard_nonbonded_force.setSwitchingDistance(switching_distance)
            custom_nonbonded_force.setUseSwitchingFunction(True)
            custom_nonbonded_force.setSwitchingDistance(switching_distance)
        else:
            standard_nonbonded_force.setUseSwitchingFunction(False)
            custom_nonbonded_force.setUseSwitchingFunction(False)
        
        self._out_system.addForce(standard_nonbonded_force)
        self._out_system_forces[standard_nonbonded_force.__class__.__name__] = standard_nonbonded_force

        self._out_system.addForce(custom_nonbonded_force)
        self._out_system_forces[custom_nonbonded_force.__class__.__name__] = custom_nonbonded_force

        
        #finally, make a custombondedforce to treat the exceptions
        custom_bonded_expression = f"(4*epsilon*((sigma/r)^12-(sigma/r)^6) + ONE_4PI_EPS0*chargeProd/r) * scale_factor; \
                                        ONE_4PI_EPS0 = {ONE_4PI_EPS0};"

        custom_bonded_expression += self.scaling_expression()

        custom_bond_force = openmm.CustomBondForce(custom_bonded_expression)
        #charges
        custom_bond_force.addPerBondParameter("chargeProd")

        #sigma
        custom_bond_force.addPerBondParameter("sigma")

        #epsilon
        custom_bond_force.addPerBondParameter("epsilon")

        #identifier
        custom_bond_force.addPerBondParameter("identifier")

        #global params
        custom_bond_force.addGlobalParameter('solute_scale', 1.0)
        custom_bond_force.addGlobalParameter('inter_scale', 1.0)    
        
        self._out_system.addForce(custom_bond_force)
        self._out_system_forces["CustomExceptionForce"] = custom_bond_force

    
    
    
    
#     def _add_nonbondeds(self):
#         self._solute_exceptions, self._interexceptions = [], []

#         #the output nonbonded force _only_ contains solvent atoms (the rest are zeroed); same with exceptions
#         """
#         First, handle the NonbondedForce in the out_system
#         """
#         og_nb_force = self._og_system_forces['NonbondedForce']
#         for particle_idx in range(self._num_particles):
#             q, sigma, epsilon = og_nb_force.getParticleParameters(particle_idx)
#             identifier = self.get_identifier(particle_idx)

#             if identifier == 1:
#                 self._out_system_forces['NonbondedForce'].addParticle(q, sigma, epsilon)
#                 self._out_system_forces['CustomNonbondedForce'].addParticle([q, sigma, epsilon, identifier])
#             else:
#                 self._out_system_forces['NonbondedForce'].addParticle(q*0.0, sigma, epsilon*0.0)
#                 self._out_system_forces['CustomNonbondedForce'].addParticle([q, sigma, epsilon, identifier])

#         #add appropriate interaction group
#         solute_ig, solvent_ig = set(self._solute_region), set(self._solvent_region)
#         self._out_system_forces['CustomNonbondedForce'].addInteractionGroup(solute_ig, solvent_ig)
#         self._out_system_forces['CustomNonbondedForce'].addInteractionGroup(solute_ig, solute_ig)

#         #handle exceptions
#         for exception_idx in range(og_nb_force.getNumExceptions()):
#             p1, p2, chargeProd, sigma, epsilon = og_nb_force.getExceptionParameters(exception_idx)
#             identifier = self.get_identifier([p1, p2])
#             if identifier == 1:
#                 self._out_system_forces['NonbondedForce'].addException(p1, p2, chargeProd, sigma, epsilon)
#                 self._out_system_forces['CustomNonbondedForce'].addExclusion(p1, p2) #maintain consistent exclusions w/ exceptions
#             elif identifier == 0:
#                 self._solute_exceptions.append([p1, p2, [chargeProd, sigma, epsilon]])
#                 self._out_system_forces['NonbondedForce'].addException(p1, p2, chargeProd*0.0, sigma, epsilon*0.0)
#                 self._out_system_forces['CustomNonbondedForce'].addExclusion(p1, p2) #maintain consistent exclusions w/ exceptions
#             elif identifier == 2:
#                 self._interexceptions.append([p1, p2, [chargeProd, sigma, epsilon]])
#                 self._out_system_forces['NonbondedForce'].addException(p1, p2, chargeProd*0.0, sigma, epsilon*0.0)
#                 self._out_system_forces['CustomNonbondedForce'].addExclusion(p1, p2) #maintain consistent exclusions w/ exceptions

#         # Add exceptions/exclusions to CustomNonbonded for inter region
#         for pair in list(itertools.product(solute_ig, solvent_ig)):
#             p1 = pair[0]
#             p2 = pair[1]
#             self._out_system_forces['NonbondedForce'].addException(p1, p2, chargeProd*0.0, sigma, epsilon*0.0)
#             self._out_system_forces['CustomNonbondedForce'].addExclusion(p1, p2)
                
#         #now add the CustomBondForce for exceptions
#         exception_force = self._out_system_forces['CustomExceptionForce']

#         for solute_exception_term in self._solute_exceptions:
#             p1, p2, [chargeProd, sigma, epsilon] = solute_exception_term
#             if (chargeProd.value_in_unit_system(unit.md_unit_system) != 0.0) or (epsilon.value_in_unit_system(unit.md_unit_system) != 0.0):
#                 identifier = 0
#                 exception_force.addBond(p1, p2, [chargeProd, sigma, epsilon, identifier])

#         for interexception_term in self._interexceptions:
#             p1, p2, [chargeProd, sigma, epsilon] = interexception_term
#             if (chargeProd.value_in_unit_system(unit.md_unit_system) != 0.0) or (epsilon.value_in_unit_system(unit.md_unit_system) != 0.0):
#                 identifier = 2
#                 exception_force.addBond(p1, p2, [chargeProd, sigma, epsilon, identifier])
        
#         # Add inter region exceptions to the CustomBondForce
#         for pair in list(itertools.product(solute_ig, solvent_ig)):
#             p1 = pair[0]
#             p2 = pair[1]
#             p1_charge, p1_sigma, p1_epsilon = og_nb_force.getParticleParameters(p1)
#             p2_charge, p2_sigma, p2_epsilon = og_nb_force.getParticleParameters(p2)
#             identifier = 2
#             exception_force.addBond(p1, p2, [p1_charge*p2_charge, 0.5*(p1_sigma+p2_sigma), np.sqrt(p1_epsilon*p2_epsilon), identifier])


In [4]:
# Create REST system
factory = REST2(ala.system, solute_region=list(range(0, 22)))
REST_system = factory.REST_system


In [5]:
# REST_system.removeForce(5)

In [7]:
REST_system.getForces()


[<simtk.openmm.openmm.MonteCarloBarostat; proxy of <Swig Object of type 'OpenMM::MonteCarloBarostat *' at 0x2ba54b032ea0> >,
 <simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2ba54b032f00> >,
 <simtk.openmm.openmm.CustomAngleForce; proxy of <Swig Object of type 'OpenMM::CustomAngleForce *' at 0x2ba54a60c2a0> >,
 <simtk.openmm.openmm.CustomTorsionForce; proxy of <Swig Object of type 'OpenMM::CustomTorsionForce *' at 0x2ba54a60c240> >,
 <simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2ba54abb12d0> >,
 <simtk.openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x2ba54abb15d0> >,
 <simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2ba54abb1900> >]

In [8]:
for i in range(REST_system.getForce(5).getNumParticles()):
    print(i, REST_system.getForce(5).getParticleParameters(i))

0 (0.1123, 0.2649532787749369, 0.06568879999999999, 0.0)
1 (-0.3662, 0.3399669508423535, 0.4577296, 0.0)
2 (0.1123, 0.2649532787749369, 0.06568879999999999, 0.0)
3 (0.1123, 0.2649532787749369, 0.06568879999999999, 0.0)
4 (0.5972, 0.3399669508423535, 0.359824, 0.0)
5 (-0.5679, 0.2959921901149463, 0.87864, 0.0)
6 (-0.4157, 0.3249998523775958, 0.7112800000000001, 0.0)
7 (0.2719, 0.10690784617684071, 0.06568879999999999, 0.0)
8 (0.0337, 0.3399669508423535, 0.4577296, 0.0)
9 (0.0823, 0.2471353044121301, 0.06568879999999999, 0.0)
10 (-0.1825, 0.3399669508423535, 0.4577296, 0.0)
11 (0.0603, 0.2649532787749369, 0.06568879999999999, 0.0)
12 (0.0603, 0.2649532787749369, 0.06568879999999999, 0.0)
13 (0.5973, 0.3399669508423535, 0.359824, 0.0)
14 (-0.5679, 0.2959921901149463, 0.87864, 0.0)
15 (0.0603, 0.2649532787749369, 0.06568879999999999, 0.0)
16 (-0.4157, 0.3249998523775958, 0.7112800000000001, 0.0)
17 (0.2719, 0.10690784617684071, 0.06568879999999999, 0.0)
18 (-0.149, 0.3399669508423535, 0.45

965 (0.417, 1.0, 0.0, 1.0)
966 (0.417, 1.0, 0.0, 1.0)
967 (-0.834, 0.3150752406575124, 0.635968, 1.0)
968 (0.417, 1.0, 0.0, 1.0)
969 (0.417, 1.0, 0.0, 1.0)
970 (-0.834, 0.3150752406575124, 0.635968, 1.0)
971 (0.417, 1.0, 0.0, 1.0)
972 (0.417, 1.0, 0.0, 1.0)
973 (-0.834, 0.3150752406575124, 0.635968, 1.0)
974 (0.417, 1.0, 0.0, 1.0)
975 (0.417, 1.0, 0.0, 1.0)
976 (-0.834, 0.3150752406575124, 0.635968, 1.0)
977 (0.417, 1.0, 0.0, 1.0)
978 (0.417, 1.0, 0.0, 1.0)
979 (-0.834, 0.3150752406575124, 0.635968, 1.0)
980 (0.417, 1.0, 0.0, 1.0)
981 (0.417, 1.0, 0.0, 1.0)
982 (-0.834, 0.3150752406575124, 0.635968, 1.0)
983 (0.417, 1.0, 0.0, 1.0)
984 (0.417, 1.0, 0.0, 1.0)
985 (-0.834, 0.3150752406575124, 0.635968, 1.0)
986 (0.417, 1.0, 0.0, 1.0)
987 (0.417, 1.0, 0.0, 1.0)
988 (-0.834, 0.3150752406575124, 0.635968, 1.0)
989 (0.417, 1.0, 0.0, 1.0)
990 (0.417, 1.0, 0.0, 1.0)
991 (-0.834, 0.3150752406575124, 0.635968, 1.0)
992 (0.417, 1.0, 0.0, 1.0)
993 (0.417, 1.0, 0.0, 1.0)
994 (-0.834, 0.3150752406575

In [7]:
for i in range(REST_system.getForce(6).getNumBonds()):
    print(i, REST_system.getForce(6).getBondParameters(i))

0 [0, 5, (-0.053145975, 0.2804727344449416, 0.12012161257658839, 0.0)]
1 [2, 5, (-0.053145975, 0.2804727344449416, 0.12012161257658839, 0.0)]
2 [3, 5, (-0.053145975, 0.2804727344449416, 0.12012161257658839, 0.0)]
3 [0, 6, (-0.03890259166666667, 0.2949765655762664, 0.10807766844265286, 0.0)]
4 [2, 6, (-0.03890259166666667, 0.2949765655762664, 0.10807766844265286, 0.0)]
5 [3, 6, (-0.03890259166666667, 0.2949765655762664, 0.10807766844265286, 0.0)]
6 [1, 7, (-0.08297481666666667, 0.2234373985095971, 0.08670021359327784, 0.0)]
7 [5, 7, (-0.128676675, 0.20145001814589353, 0.12012161257658839, 0.0)]
8 [1, 8, (-0.010284116666666667, 0.3399669508423535, 0.2288648, 0.0)]
9 [5, 8, (-0.015948525, 0.3179795704786499, 0.3170881327580709, 0.0)]
10 [4, 9, (0.040957966666666665, 0.2935511276272418, 0.07687068162049819, 0.0)]
11 [7, 9, (0.01864780833333333, 0.17702157529448542, 0.032844399999999996, 0.0)]
12 [4, 10, (-0.09082416666666666, 0.3399669508423535, 0.20291752979375635, 0.0)]
13 [7, 10, (-0.04

In [5]:
T_min = 298.0 *unit.kelvin
# T = 600.0 * unit.kelvin
T = 298*unit.kelvin

# Create thermodynamic state
lambda_zero_alchemical_state = RESTState.from_system(REST_system)
thermostate = ThermodynamicState(REST_system, temperature=T_min)
compound_thermodynamic_state = CompoundThermodynamicState(thermostate,
                                                          composable_states=[lambda_zero_alchemical_state])

# Set alchemical parameters
beta_0 = 1 / (kB * T_min)
beta_m = 1 / (kB * T)
compound_thermodynamic_state.set_alchemical_parameters(beta_0, beta_m)

# # Create sampler state
# sampler_state = SamplerState(positions, box_vectors=REST_system.getDefaultPeriodicBoxVectors())

# # Relax and save energy
# # context_cache = cache.ContextCache()
# # context, context_integrator = context_cache.get_context(compound_thermodynamic_state)
# feptasks.minimize(compound_thermodynamic_state, sampler_state)
# REST_energy = compound_thermodynamic_state.reduced_potential(sampler_state)

integrator = openmm.VerletIntegrator(1.0*unit.femtosecond)
context = compound_thermodynamic_state.create_context(integrator)
context.setPositions(ala.positions)
sampler_state = SamplerState.from_context(context)
compound_thermodynamic_state.reduced_potential(sampler_state)


-9823.53269381644

In [7]:
system = ala.system
# system = toy_system

# Compute energy for non-RESTified system
# Determine regions and scaling factors
# protein = range(6, 16)
# environment = list(range(0, 6)) + list(range(16, 22))
# protein = list(range(0, 22))
# environment = list(range(22, system.getNumParticles()))
# protein_scaling = beta_m / beta_0
# inter_scaling = np.sqrt(beta_m / beta_0)


# # Scale the terms in the bond force appropriately
# bond_force = system.getForce(0)
# for bond in range(bond_force.getNumBonds()):
#     p1, p2, length, k = bond_force.getBondParameters(bond)
#     if p1 in protein and p2 in protein:
#         bond_force.setBondParameters(bond, p1, p2, length, k * protein_scaling)
#     elif (p1 in protein and p2 in environment) or (p1 in environment and p2 in protein):
#         bond_force.setBondParameters(bond, p1, p2, length, k * inter_scaling)

# # Scale the terms in the angle force appropriately
# angle_force = system.getForce(1)
# for angle_index in range(angle_force.getNumAngles()):
#     p1, p2, p3, angle, k = angle_force.getAngleParameters(angle_index)
#     if p1 in protein and p2 in protein and p3 in protein:
#         angle_force.setAngleParameters(angle_index, p1, p2, p3, angle, k * protein_scaling)
#     elif set([p1, p2, p3]).intersection(set(protein)) != set() and set([p1, p2, p3]).intersection(set(environment)) != set():
#         angle_force.setAngleParameters(angle_index, p1, p2, p3, angle, k * inter_scaling)

# # Scale the terms in the torsion force appropriately
# torsion_force = system.getForce(2)
# for torsion_index in range(torsion_force.getNumTorsions()):
#     p1, p2, p3, p4, periodicity, phase, k = torsion_force.getTorsionParameters(torsion_index)
#     if p1 in protein and p2 in protein and p3 in protein and p4 in protein:
#         torsion_force.setTorsionParameters(torsion_index, p1, p2, p3, p4, periodicity, phase, k * protein_scaling)
#     elif set([p1, p2, p3, p4]).intersection(set(protein)) != set() and set([p1, p2, p3, p4]).intersection(set(environment)) != set():
#         torsion_force.setTorsionParameters(torsion_index, p1, p2, p3, p4, periodicity, phase, k * inter_scaling)

# protein_scaling = 0
# inter_scaling = 0
        
# Scale the terms in the nonbonded force appropriately
# nb_force = system.getForce(3)
# for nb_index in range(nb_force.getNumParticles()):
#     charge, sigma, epsilon = nb_force.getParticleParameters(nb_index)
#     if nb_index in protein:
#         nb_force.setParticleParameters(nb_index, protein_scaling * charge, sigma, protein_scaling * epsilon)

# protein_scaling = 0
# inter_scaling = 0
# protein_scaling = beta_m / beta_0
# inter_scaling = np.sqrt(beta_m / beta_0)

# # Scale the exceptions in the nonbonded force appropriately
# nb_force = system.getForce(3)
# for nb_index in range(nb_force.getNumExceptions()):
#     p1, p2, chargeProd, sigma, epsilon = nb_force.getExceptionParameters(nb_index)
#     if p1 in protein and p2 in protein:
#         nb_force.setExceptionParameters(nb_index, p1, p2, protein_scaling * chargeProd, sigma, protein_scaling * epsilon)
#     elif (p1 in protein and p2 in environment) or (p1 in environment and p2 in protein):
#         nb_force.setExceptionParameters(nb_index, p1, p2, inter_scaling * chargeProd, sigma, inter_scaling * epsilon)

# exception_pairs = [tuple(sorted([nb_force.getExceptionParameters(nb_index)[0], nb_force.getExceptionParameters(nb_index)[1]])) for nb_index in range(nb_force.getNumExceptions())]
# solute_pairs = set([tuple(sorted(pair)) for pair in list(itertools.product(protein, protein))])
# # Scale nonbonded interactions for solute region by adding exceptions for all pairs of atoms 
# for pair in list(solute_pairs):
#     p1 = pair[0]
#     p2 = pair[1]
#     p1_charge, p1_sigma, p1_epsilon = nb_force.getParticleParameters(p1)
#     p2_charge, p2_sigma, p2_epsilon = nb_force.getParticleParameters(p2)
#     if p1 != p2:
#         if pair not in exception_pairs:
# #             print(pair)
#             nb_force.addException(p1, p2, p1_charge*p2_charge*protein_scaling, 0.5*(p1_sigma+p2_sigma), np.sqrt(p1_epsilon*p2_epsilon)*protein_scaling)
        
# # Scale nonbonded interactions for inter region by adding exceptions for all pairs of atoms 
# # traj = md.Trajectory(np.array(positions), md.Topology.from_openmm(solvated_topology))
# # nearby_waters = md.compute_neighbors(traj, 0.4, list(range(0,22)), haystack_indices=list(range(22, solvated_topology.getNumAtoms())))[0]
# for pair in list(itertools.product(protein, environment)):
#     p1 = pair[0]
#     p2 = int(pair[1]) # otherwise, will be a numpy int
#     p1_charge, p1_sigma, p1_epsilon = nb_force.getParticleParameters(p1)
#     p2_charge, p2_sigma, p2_epsilon = nb_force.getParticleParameters(p2)
#     nb_force.addException(p1, p2, p1_charge*p2_charge*inter_scaling, 0.5*(p1_sigma+p2_sigma), np.sqrt(p1_epsilon*p2_epsilon)*inter_scaling)

# # Save energy
# thermostate = ThermodynamicState(system, temperature=T_min)
# # context_cache = cache.ContextCache()
# # context, context_integrator = context_cache.get_context(thermostate)
# nonREST_energy = thermostate.reduced_potential(sampler_state)

thermostate = ThermodynamicState(system, temperature=T_min)
integrator = openmm.VerletIntegrator(1.0*unit.femtosecond)
context = thermostate.create_context(integrator)
context.setPositions(ala.positions)
sampler_state = SamplerState.from_context(context)
thermostate.reduced_potential(sampler_state)

-9954.18985914795